# 🧩 Introduction
Welcome to the GenAI Playground for Kids! This interactive notebook is designed to introduce children aged 8–14 to the fascinating world of Generative AI through engaging activities like storytelling, drawing, and Q&A sessions.

# 🎯 Objective
The objective of this Capstone Project is to create a fun, interactive AI-powered playground for children aged 8–12 using Google’s Gemini models. This project aims to introduce young learners to generative AI in a safe and creative way by allowing them to:

    - Turn simple story ideas into complete, imaginative stories using few-shot prompting
    
    - Visualize their stories through AI-generated images
    
    - Listen to their stories in fun, engaging voices using AI-generated audio

The goal is to make learning with AI exciting, accessible, and inspiring for the next generation of creators and explorers.

# 👨‍🔬 Target Audience
### Primary: Children aged 8–14.

# 🛠️ Setup
Let's start by installing and importing the necessary libraries.

In [ ]:
# Uninstall everything related to async framework
!pip uninstall -y anyio gradio starlette fastapi

# Reinstall exact version of google-generativeai and its correct dependency
!pip install -q google-generativeai==0.8.5 google-ai-generativelanguage==0.6.15

# Now install langgraph and compatible langchain-google-genai version
!pip install -qU jupyterlab jupyterlab-lsp langchain-google-genai==2.1.2 langchain-google-community google-search-results

# Install specific components
!pip install anyio==3.7.1 fastapi==0.110.0 starlette==0.36.3 gradio pyttsx3

# Imports
import gradio as gr
import google.generativeai as genai
import PIL
from PIL import Image
import IPython.display as display
import os
import requests
import io
import pyttsx3 


from google import genai
from google.genai import types

from IPython.display import Markdown, HTML, display

print(genai.__version__)

### Set up your API key

To run the following cell, your API key must be stored it in a [Kaggle secret](https://www.kaggle.com/discussions/product-feedback/114053) named `GOOGLE_API_KEY`.

If you don't already have an API key, you can grab one from [AI Studio](https://aistudio.google.com/app/apikey). You can find [detailed instructions in the docs](https://ai.google.dev/gemini-api/docs/api-key).

To make the key available through Kaggle secrets, choose `Secrets` from the `Add-ons` menu and follow the instructions to add your key or enable it for this notebook.

In [ ]:
from kaggle_secrets import UserSecretsClient

GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")

client = genai.Client(api_key=GOOGLE_API_KEY)

# Example: Text generation using the model
response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents="Explain AI to me like I'm a kid.")

print(response.text)

📖 Story Generator
Let's create a function that generates a story based on a prompt provided by the user.

In [ ]:
# --- Story Generator with Few-Shot Prompting ---
def generate_story(prompt):
    few_shot_examples = [
        {"role": "user", "content": "Once upon a time, a dragon baked a giant cake in the forest."},
        {"role": "user", "content": "Lily the explorer found a map that led to a land of candy."},
        {"role": "user", "content": "A brave puppy flew a rocket to the moon to rescue his robot friend."}
    ]
    # Add the user's fun story idea
    combined_prompt = few_shot_examples + [{"role": "user", "content": prompt}]
    
    response = client.models.generate_content(
        model="gemini-2.0-pro",  # Use gemini-2.0 or gemini-2.0-pro
        contents=combined_prompt
    )
    return response.text  # Assuming text is directly returned

# --- Generate Image from Story ---
def generate_image_from_story(story_text):
    image_response = client.models.generate_content(
        model="gemini-2.0-pro",
        contents=[{"role": "user", "content": f"Draw a fun cartoon-style image for this story: {story_text}"}]
    )
    return image_response.text  # Assuming a URL or path to image

# --- Convert Story to Fun Audio (Text to Speech) ---
def generate_audio_from_story(story_text):
    audio_response = client.models.generate_content(
        model="gemini-2.0-pro",
        contents=[{"role": "user", "content": f"Read this story in a fun voice for kids: {story_text}"}]
    )
    return audio_response.text  # Assuming an audio URL or base64 audio

# --- Gradio Interface for Kids ---
with gr.Blocks() as demo:
    gr.Markdown("<h1 style='text-align: center;'>🎨 GenAI Playground for Kids 🎤</h1><p style='text-align: center;'>Tell a story idea, and we'll turn it into a story, picture, and voice!</p>")

    with gr.Tab("🌈 Create Your Story"):
        story_input = gr.Textbox(label="📝 Your Story Idea", placeholder="e.g. A dog who becomes a superhero!", lines=2)
        story_output = gr.Textbox(label="📖 Your Full Story")
        story_image = gr.Image(label="🎨 Your Story Picture", type="filepath")
        story_audio = gr.Audio(label="🎤 Hear Your Story")

        def run_story_pipeline(prompt):
            story = generate_story(prompt)
            image = generate_image_from_story(story)
            audio = generate_audio_from_story(story)
            return story, image, audio

        generate_btn = gr.Button("✨ Make My Story!")
        generate_btn.click(fn=run_story_pipeline, inputs=story_input, outputs=[story_output, story_image, story_audio])

demo.launch()


🧠 Conclusion
The GenAI Playground for Kids is a novel educational tool that merges the excitement of Generative AI with the simplicity needed for early learners. Through interactive storytelling, image creation, and playful Q&A, it encourages children to understand and appreciate AI in a safe and engaging way.